# Week 6-1: Election Prediction through simulation

This is the first of two classes on election prediction. We'll be using simulation throughout to build our model.

All data is downloaded from [Huffington Post Pollster](http://elections.huffingtonpost.com/pollster#historical-charts)

Further references for your enjoyment:

- [The Real Story of 2016](https://fivethirtyeight.com/features/the-real-story-of-2016/) - Fivethirtyeight
- Buzzfeed's post-election [forecast grades](https://www.buzzfeednews.com/article/jsvine/2016-election-forecast-grades)
- [Putting the Polling Miss of the 2016 Election in Perspective](https://www.nytimes.com/interactive/2016/11/13/upshot/putting-the-polling-miss-of-2016-in-perspective.html) - The Upshot
- [After 2016, Can We Ever Trust the Polls Again?](https://newrepublic.com/article/139158/2016-can-ever-trust-polls-again) - The New Republic 

And finally, the single biggest reason that the simple election prediction model in this file misses so badly (predicting Clinton's chances in the high 90s): it does not take into account the [correlations between polling errors](https://www.quantamagazine.org/why-nate-silver-sam-wang-and-everyone-else-were-wrong-part-2-20161111/) in different states. If we fix this one factor, even our simple model will give Trump substantially higher chances.

## Part 1: Simulating one poll

Here we'll produce simulated election outcomes from a single poll. We are uncritically taking the poll results as an unbiased inidicator of results. This assumes that the people who are polled ("likely voters") are a good representation of the people who actually vote. It is possible to adjust these sorts of factors later, but let's begin with the basics.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Set a random seed so this whole notebook becomes deterministic (for teaching purposes)


In [ ]:
# Load national polling data. It's a TSV file, so we have to tell read_csv it's separated by tabs


In [ ]:
# keep only polls of "likely voters" (as opposed to registered voters, or republicans/democracts)


In [ ]:
# We need a margin of error to do our simulation, so drop any rows that don't have it


There are lot of polls here! Let's pick just one, near the end of the polling period (just before the election) and look at the outcomes it implies.

We will pick a poll which showed a close race, because it better demonstrates how the margin of error works.

The poll results and the margin of error define a probability distribution of "true" survey results -- that is, the result that the pollster would get if they could ask every single "likely voter" in the country. This distribution is a "normal" distribution.

In [ ]:
# We reduce the problem to the difference between the two poll results, because that's what actually matters
mean = poll.Clinton - poll.Trump

# Some subtlety in calculating the stddev from the margin of error (MOE)
#  - MOE is reported as 95% width, so we'd normally divide by 1.96 for standard deviation
#  - But we want the stddev for a difference between two poll questions which are not independent.
#    One more vote for Clinton is (almost always) one less vote for Trump. We need to multiply by nearly 2.
#  - These almost perfectly cancel out, and stddev of the difference is near exactly MOE
# See http://abcnews.go.com/images/PollingUnit/MOEFranklin.pdf
stddev = poll.margin_of_error

# For more general discussion of the MOE see 
# http://www.pewresearch.org/fact-tank/2016/09/08/understanding-the-margin-of-error-in-election-polls/


Now we can take samples from a normal distribution with this mean and standard deviation, to simulate what the underlying "true" voting pattern would be. For example,

To interpret this number, recall that we're simulating the Clinton-Trump difference. So positive means it goes for Clinton, who our poll says is ahead 46-44. Given this, we would expect more of the simulation results to go for Clinton. Let's make 1000 and see what happens.

Sure enough, the center of this distribution is at 2, the lead given by the polls. But many values are negative as well, meaning that Clinton doesn't always win (again, assuming the actual voters splt 46-44, as this poll suggests.)

Let's see how often Trump wins, according to this model

So about 24% according to this model. This makes sense becuase the margin of error (2.8%) is pretty wide relative to the difference between the polls (2%) 

If we run the simulation again, we'll get slightly different results.

The more samples we take, the less variation we'll see in this number. To demonstrate this, let's plot a histogram of the results for various numbers of samples.

In [ ]:
plt.hist(np.random.normal(mean, stddev, 100), bins=20, density=True)
plt.title('100 samples')
plt.show()
plt.hist(np.random.normal(mean, stddev, 1000), bins=20, density=True)
plt.title('1,000 samples')
plt.show()
plt.hist(np.random.normal(mean, stddev, 10000), bins=20, density=True)
plt.title('10,000 samples')
plt.show()
plt.hist(np.random.normal(mean, stddev, 100000), bins=20, density=True)
plt.title('100,000 samples')
plt.show()

We can get a reliable win percentage by counting the wins in a large sample:

## Part 2: The Electoral college
This shows how to interpret the uncertainty in a single poll -- at least the uncertainty in the margin of error. There are two major directions to go from here:

1) In a real election prediction model, we would combine all the polls according to a weighted average of poll reliability. What "reliability" really means is how well the poll matched (predicted) previous election results. We can figure out the optimal combination of poll weights using, for example, linear regression.

2) The US uses an electoral college system, where each state contributes a fixed number of votes (out of a total of 538). We definitely need to simulate that to get anything like a reasonable election prediction.

So for the next step, let's see how to combine polls in the electoral college.

Our first task will be to pick out one poll in each state. We'll use the last dated "Likely Voter" poll.

In [ ]:
# Load a CSV of electoral college votes for each state. 
# Ref: https://www.archives.gov/federal-register/electoral-college/allocation.html


In [ ]:
# We'll use a little Pandas trick to make merging in the poll data easier: 
# set the index to the abbreviation

# And add the columns we'll need: Trump, Clinton, margin_of_error, all initially blank


In [ ]:
# Not all polls have reported margins of error, but we can figure it out if we know the number of people surveyed.
# This function salculate the 95% margin of error, using the classic formula. 
# Ref: https://onlinecourses.science.psu.edu/stat100/node/56/


In [ ]:
# Now we'll load polls for each state and pick one poll
for abbr in states.abbr: 
    polls = pd.read_csv('data/' + abbr + '.tsv', sep='\t')
    polls = polls[polls.sample_subpopulation == 'Likely Voters']
    poll = polls.tail(1).squeeze() 

    states.loc[abbr,'Trump'] = poll.Trump
    states.loc[abbr,'Clinton'] = poll.Clinton

    # There may be no MOE reported for this poll. If not, calculate it
    moe = poll.margin_of_error
    if pd.isnull(moe):
        proportion = poll.Trump / 100  # or Clinton, will give nearly same result 
        moe = calc_moe(poll.observations, proportion)

    states.loc[abbr,'margin_of_error'] = moe

In [ ]:
states.head()

Now we simulate an election by drawing a sample from each state election indpendently, then tallying the electoral college votes. Instead of looking at the distribution of Clinton-Trump vote, we'll just look at the distribution of EC votes for Clinton.

In [ ]:
def simulate_election(n_times):

In [ ]:
# Run 10 simulated elections and look at the results


In [ ]:
# Run many, many simulated elections and plot histogram of results


To get a Clinton win probability out of this, we can calculate the percentage where she receives 270 or more.

## Part 3: Correlated errors

In [ ]:
# Let's start with some random numbers
mean=0
stddev=1
n=10
np.random.normal(mean, stddev, n)

In [ ]:
# Sum of these random numbers
np.random.normal(mean, stddev, n).sum()

In [ ]:
def plot_distribution_of_sums(make_a_sum_function, n_times):
    sums = pd.DataFrame(np.zeros(n_times))
    sums = sums.applymap(make_a_sum_function)
    sums.plot(kind='hist', bins=20)
    print("standard deviation")
    print(float(sums.std()))
    

In [ ]:
# If take the sum of these random numbers 1000 times, what do we get?


But suppose that half of these random numbers were actually the *same* random number...

## Part 4: Elections with correlated errors

A poll is meant to tell us how people will vote in an election. But there will be some difference between the last latest polls before an election and the actual election results. According to [this research](http://www.stat.columbia.edu/~gelman/research/unpublished/polling-errors.pdf), US state level presidentail polls in the last three weeks before an election are off by an average of 2%.

So we could simply add 2% to our margins of error. But this isn't quite right: when a poll is off in one state, it's often off in other states for similar reasons. The polling error is *correlated.* Not taking into account correlated polling errors were the [biggest reason](https://fivethirtyeight.com/features/election-update-why-our-model-is-more-bullish-than-others-on-trump/) that many 2016 election predictions were so badly off.

First, let's see what simply doubling the margin of error on every state does. This increases error, but not *correlated* error.

In [ ]:
# Helper function to interpret results
def plot_results(results):
    plt.hist(results, bins=range(150, 500, 10), density=True)
    plt.axvline(270, color='black', linestyle='dashed');
    print("Clinton win probability: " + str((results>=270).mean()))

Instead, we need to add polling error that is similar between states. The simplest way to do this is just to add the same polling error to every state (perfectly correlated across all states!)

In [ ]:
def simulate_election_national_error(n_times, polling_error_stddev):


In [ ]:
# What does the distribution of electoral college outcomes look like with 2% correlated national polling error?
